# Transform a Bus Route Polyline into a Route Layer

It’s summer in Portland, Maine, and two local schools have partnered up to offer students a week long camping trip to Peaks Island. King Middle School and Jack Middle School students will board a bus at their respective schools, and then they will be driven to the Peaks Island Ferry for drop-off. You, as the school bus dispatcher, are to create the bus routes the two bus drivers will follow. 

In this lesson, you’ll take two existing bus routes that were generated using a 3rd party bus scheduling solution and convert them into route layers that can be used in Navigator for ArcGIS.Navigator will provide bus drivers with turn-by-turn directions, even in a disconnected environment.

You will use the **ArcPy** module to automate this workflow. 


**Lesson Plan:**

1.	Prepare stops layer 
2.	Create route analysis layer and solve routes
3.	Share route layers


## Prepare stops layer

You will first prepare the stops layer that will be used to generate bus routes to and from each school. You will take the vertices that were used to create the bus route polylines and convert them into a point feature layer that can then be imported into a route. By deriving this stops layer from the polyline geometry, you are maintaining the integrity of the original routes you created using the 3rd party bus scheduling solution. 

**1. Connect to your workspace**

The first thing you'll do is import the **os module** to connect to the local drive that contains the project data. You'll then import the **ArcPy module**. This is the Python site package for ArcGIS that allows you to manipulate geographic data and automate workflows. Find the complete documentation for ArcPy [here](http://desktop.arcgis.com/en/arcmap/10.3/analyze/arcpy/what-is-arcpy-.htm).

In [85]:
import arcpy 
import os 

## establish a directory to the local drive that is storing the data for this project 
notebook_dir = os.path.abspath(r'C:\Users\jos10367\Documents\GitHub\navigator-scripts\data\SummerCampBusRoutes')

## overwrite environment to make sure that previous outputs made within this notebook are overwritten 
arcpy.env.overwriteOutput = True

notebook_dir

'C:\\Users\\jos10367\\Documents\\GitHub\\navigator-scripts\\data\\SummerCampBusRoutes'

**2. Simplify your polylines**

The more complex a polyline is, the more vertices it will have. Since the vertices of these lines will be used as stops and waypoints to guide your route creation, you'll want to limit the amount you're working with. The [simplify line](https://pro.arcgis.com/en/pro-app/tool-reference/cartography/simplify-line.htm) tool allows you to to remove extraneous vertices of a polyline while preserving its essential shape. 

You'll run this tool to simplify your bus route polylines. 

In [86]:
import arcpy.cartography as CA

bus_lines = os.path.join(notebook_dir,'SummerCampBusRoutes.gdb/BusRoutes')
output_file = os.path.join(notebook_dir, 'SummerCampBusRoutes.gdb/SimplifiedBusRoutes')

#POINT_REMOVE retains critical points that preserve the essential shape of the line

#The tolerance we will use is 25 meters. This is the maximum allowable distance between each vertex and the new line.

simplified_bus = CA.SimplifyLine(bus_lines,output_file,"POINT_REMOVE",25)

**3. Convert simplified polyline vertices into point feature data**

This **Feature Vertices to Points** tool will take the vertices of your simplified polylines and convert them into point feature data. These points will be serve as the **stops and waypoints** that will solve the final route layers. 

In [87]:
output_file = os.path.join(notebook_dir, 'SummerCampBusRoutes.gdb\BusPoints')

bus_points = arcpy.FeatureVerticesToPoints_management(simplified_bus, output_file, "ALL")

## Create route analysis layer and solve routes

It’s time to generate your routes. In the previous lesson, you prepared a point feature layer from the vertices of your polylines. Now, you will create a route layer using the Network Analysis extension. Once created, you will import your point feature layer in as stops and then solve the route. 

**1. Create route analysis layer**

Now you will set up your **route analysis layer**. This is used for determining the best route between a set of locations. You are able to use your own network data source for this, but for this project you'll just be pulling the network from ArcGIS Online. 

In [88]:
arcpy.na.MakeRouteAnalysisLayer("https://www.arcgis.com/", #network data source
                                "Route",                   #route layer name
                                "Driving Time",            #travel mode
                                "USE_CURRENT_ORDER",       #sequence
                                 None,                     #time of day (not needed for this lesson)
                                "LOCAL_TIME_AT_LOCATIONS", #time_zone (not needed) 
                                "ALONG_NETWORK",           #line_shape
                                 None,                     #accumulate attributes (not needed)
                                "DIRECTIONS")              #generates directions

<Result 'Route'>

**2. Import Stops**

You will now **import your bus points layer as stops** into your newly created route. You will utilize a **field mapping** parameter to ensure that two unique routes will be created based off of the RouteName field of your bus points layer (either King Middle or Jack Jr. High). You will **sort your stops** using **OBJECTID** as they this correlates with the direction in which you want your routes to go. 

Complete documentation for the **add locations** tool can be found [here](https://pro.arcgis.com/en/pro-app/tool-reference/network-analyst/add-locations.htm).

In [89]:
arcpy.na.AddLocations("Route",                     #route layer you are adding stops to
                      "Stops",                     #layer type you are adding
                      bus_points,                  #points you are importing as stops
                      "Name # #;RouteName RouteName #;Sequence # #;TimeWindowStart # #;TimeWindowEnd # #;LocationType # 0;CurbApproach # 0;Attr_Minutes # 0;Attr_TravelTime # 0;Attr_Miles # 0;Attr_Kilometers # 0;Attr_TimeAt1KPH # 0;Attr_WalkTime # 0;Attr_TruckMinutes # 0;Attr_TruckTravelTime # 0",
                      "5000 Meters",               #search tolerance used to locate the input features on the network
                      "OBJECTID")                  #the field on which the network analysis objects are sorted


<Result 'Route'>

**3. Convert intermediate stops into waypoints**

A stop within a route layer can have one of three location types, each with an assigned value.

|Location Type|Description|Value|
|-|-|-|
|Stop|A location that the route should visit. This is the default| 0 |
|Waypoint|A location that the route should travel through without making a stop.| 1 | 
|Break|A location where the route stops for the driver to take a break| 2 | 

Start by setting your workspace within the SummerCampBusRoutes geodatabase. This will make it easier when calling out the layers that you'll be working with. 

In [90]:
arcpy.env.workspace = os.path.join(notebook_dir, 'SummerCampBusRoutes.gdb')

Get the total count of stops within the Stops sublayer of your Route. You will use this to find the number of stops for each inidividual route within the next step.

In [91]:
results = arcpy.GetCount_management("Stops")
results

<Result '21'>

In this next step, you will iterate through your stops layer in order to find out how many stops are in each route. You will store these values in **king_count** and **jack_count**. Print these results so you know how many stops each route has.

In [92]:
king_count = 0
jack_count = 0 

cursor = arcpy.da.SearchCursor("Stops", ['RouteName'])
for row in cursor:
     if row[0] == 'King Middle School':
            king_count = king_count + 1
     else:
            jack_count = jack_count + 1

king_count = int(king_count)
jack_count = int(jack_count)

print(king_count)
print(jack_count)

10
11


Now you will use the **SelectLAyerByAttribute** tool to select the stops you want to **preserve** as stops. Remember that we will be switching this selection in the next step, so that we can turn every other stop into a **waypoint**. 

In [93]:
##selecting the first stop in each unique route

arcpy.management.SelectLayerByAttribute("Stops", "ADD_TO_SELECTION", 'Sequence=1')

#selecting the last stop in the King Middle School Route

arcpy.management.SelectLayerByAttribute("Stops", "ADD_TO_SELECTION", 
                                        "RouteName = 'King Middle School' And Sequence = " + str(king_count)) 

#selecting the last stop in the Jack Jr. Highschool Route

arcpy.management.SelectLayerByAttribute("Stops", "ADD_TO_SELECTION", 
                                        "RouteName = 'Jack Middle School' And Sequence = " + str(jack_count)) 

<Result 'Stops'>

Go ahead and check that you've selected the stops you want to preserve by listing them with a cursor. The following code should print four stops total; 2 from each middle school route. 

In [94]:
cursor = arcpy.da.SearchCursor("Stops", ['Sequence', 'RouteName'])
for row in cursor:
    print(row)

(1, 'King Middle School')
(10, 'King Middle School')
(1, 'Jack Jr. High')
(11, 'Jack Jr. High')


Use the **Switch Selection** parameter in the **Select Layer by Attribute** tool to switch your selection from these four stops to every other stop. Then use the **Calculate Field** tool to change the location type of the selected stops,currently valued at 0,to the waypoint value of 1. 

In [95]:
arcpy.management.SelectLayerByAttribute("Stops","SWITCH_SELECTION")

arcpy.management.CalculateField("Stops", "LocationType", "1", "PYTHON3", '')

<Result 'Stops'>

Clear your selection and print your updated stops layer to ensure that you have a stop (0) at the beginning and end of each route and that you only have waypoints (1) in-between. 

In [96]:
arcpy.management.SelectLayerByAttribute("Stops", "CLEAR_SELECTION")

cursor = arcpy.da.SearchCursor("Stops", ['Sequence', 'RouteName','LocationType'])
for row in cursor:
    print(row)

(1, 'King Middle School', 0)
(2, 'King Middle School', 1)
(3, 'King Middle School', 1)
(4, 'King Middle School', 1)
(5, 'King Middle School', 1)
(6, 'King Middle School', 1)
(7, 'King Middle School', 1)
(8, 'King Middle School', 1)
(9, 'King Middle School', 1)
(10, 'King Middle School', 0)
(1, 'Jack Jr. High', 0)
(2, 'Jack Jr. High', 1)
(3, 'Jack Jr. High', 1)
(4, 'Jack Jr. High', 1)
(5, 'Jack Jr. High', 1)
(6, 'Jack Jr. High', 1)
(7, 'Jack Jr. High', 1)
(8, 'Jack Jr. High', 1)
(9, 'Jack Jr. High', 1)
(10, 'Jack Jr. High', 1)
(11, 'Jack Jr. High', 0)


**4. Solve route analysis layer**

You will now run the Solve the route analysis layer. Two unique routes will be generated along with turn-by-turn directions. 

In [97]:
arcpy.na.Solve("Route")


<Result 'Route'>

## Share route layers

Now that you have created and solved the bus routes, you are ready to share them with your organization. You will use the **Share As Route Layers** tool to share the bus routes with your bus drivers. They will now have turn by turn directions available to them when using Navigator for ArcGIS. 

**1. Share routes to ArcGIS Online**

The **ShareAsRouteLayers** tool will share your two bus routes to your ArcGIS Online account. If you received an error message that reads **'ToolValidator' object has no attribute 'isLicensed'**, you can ignore it. It has no effect on this workflow. 

In [98]:
route_layer = arcpy.na.ShareAsRouteLayers("Route",     #input route
                            "Summer Camp bus routes",  #summary
                            "busroutes, summercamp",   #tags
                            "Peaks Ferry Bus Route",   #a prefix for each unique route created
                            "josh_nitro",              #your user name
                            "MYCONTENT",               #where you want your routes to appear
                             None)                     #if you want to add routes to a specific folder within AGOL


AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

**2. Create Navigator Links**

By creating unique **links** for each route, you are able to share it with its respective driver. These links will open up to each bus route on Navigator when clicked on your mobile device.

First you will **set the number of routes you have**. You have 2 routes, so you want to make sure you are creating two unique links. You will then want to obtain a map item ID. This will determine what mobible map package (mmpk) your routes open up with in Navigator. An item id for Southern Maine has been provided for this exercise.

In [99]:
num_routes = 0
cursor = arcpy.da.SearchCursor("Stops", [ 'Sequence'])
for row in cursor:
    if row[0] == 1:
        num_routes = num_routes + 1
num_routes

2

In [100]:
map_item = '69a77ed166044909b8f7469dd4f31b84' #this is the item ID for a map of Southern Maine

From the apps module, import the Navigator URL builder function and then create a unique URL for the route result

 - a. create url_type of 'Web' that can be opened on iOS or Android devices from email, text message, etc.
 - b. set theroute_item parameter from the route's RouteLayerItemID attribute
 - c. use the preferred map as the webmap parameter (map_item)

In [101]:
from arcgis.apps import build_navigator_url

route_string = str(route_layer)        #turns your results layer into a string
route_unique = route_string.split(",") #parses the string into each individual route result 


n=0 

while n < num_routes: 
            route_layer_item_id = (route_unique[n].split("="))[1].split("'")[0] #parse url to get itemID
            url=build_navigator_url(url_type='Web', webmap=map_item, route_item=route_layer_item_id)
            print(url)
            n = n+1

https://navigator.arcgis.app?itemID=69a77ed166044909b8f7469dd4f31b84&routeItemID=d2781c9925b241c481585b0ffb8808fc
https://navigator.arcgis.app?itemID=69a77ed166044909b8f7469dd4f31b84&routeItemID=98cbed7364ef42c48248d42d7985db8e


Clicking on either of these links on your mobile device will opn its respective route in Navigator! 